# Segmenting and Clustering Neighborhoods in Toronto

## Part 1 - Scraping wiki page and get the lists of neighborhood in Toronto 

In [ ]:
!conda install -c anaconda beautifulsoup4

In [3]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.



In [4]:
!pip install geocoder

In [1]:
#import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import geocoder 
import folium

In [7]:
# requests data from wiki ppage and create the dataframe
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r=requests.get(url)
#print(r)
soup=BeautifulSoup(r.text,'html.parser')
#print(soup)

table=soup.find_all('table')[0]
#print(table)
all_postalcode=table.find_all('p')
#print(len(all_postalcode))
data=[]

PostalCode=[]
Borough=[]
Neighborhood=[]
for p in all_postalcode:
    
    PCode=p.find('b').text
    #print(PCode)
    Borough_i=p.text.replace(PCode,'')
    #print(Borough_i)
    if Borough_i=="Not assigned\n":
        pass
    else:
        PostalCode=PCode
        Borough_Nei=Borough_i.split('(')
        #print(Borough_Nei)
        Borough=Borough_Nei[0]
        #print(Borough)
        Neighborhood=Borough_Nei[1].replace(')\n','')
        Neighborhood=Neighborhood.replace('\n','')
        Neighborhood=Neighborhood.split(')')[0]
        #print(PostalCode, Borough, Neighborhood)      
        data.append([PostalCode,Borough,Neighborhood])
#print(data)
df=pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood'])
#print(df[df['PostalCode']=='M3N'])
df=df.drop_duplicates()
print(df.shape)
df

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


## Part 2 - get the latitude and longitude of all the neighborhoods

get the latitude and longitude of each borough of Toronto

In [15]:
borough_list=df['Borough'].values

Latitude=[]
Longitude=[]


for borough_each in borough_list:
    #print(borough_each)
    try:
        geolocator = Nominatim(user_agent="toronto_explorer")

        g=geolocator.geocode('{}, Toronto, Ontario'.format(borough_each))
        #print("g is {}".format(g) )
      
                      
        Latitude.append(g.latitude)
        Longitude.append(g.longitude)
    except:
        Latitude.append(0)
        Longitude.append(0)
    
        
#print(Latitude,Longitude)
df_complete=df.copy()

df_complete['Latitude']=Latitude
df_complete['Longitude']=Longitude
df_complete

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.754326,-79.449117
1,M4A,North York,Victoria Village,43.754326,-79.449117
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.656322,-79.380916
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.754326,-79.449117
4,M7A,Queen's Park,Ontario Provincial Government,43.659659,-79.390340
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.643556,-79.565633
99,M4Y,Downtown Toronto,Church and Wellesley,43.656322,-79.380916
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,0.000000,0.000000
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.643556,-79.565633


clean the data, remove one borough that has the wrong latitude and longitude (which shows 0)

In [17]:
df_complete[df_complete['Latitude']==0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
35,M4J,East YorkEast Toronto,The Danforth East,0.0,0.0
76,M7R,MississaugaCanada Post Gateway Processing Centre,Enclave of L4W,0.0,0.0
92,M5W,Downtown TorontoStn A PO Boxes25 The Esplanade,Enclave of M5E,0.0,0.0
94,M9W,EtobicokeNorthwest,Clairville / Humberwood / Woodbine Downs / Wes...,0.0,0.0
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,0.0,0.0


In [19]:
df_complete=df_complete[df_complete['Latitude']!=0]
df_complete.shape

(98, 5)

explode each neighborhood a row

In [23]:
df_new_1=df_complete.copy()
df_new_1['Neighborhood']=df_new_1['Neighborhood'].str.split('/')
df_final=df_new_1.explode('Neighborhood').reset_index(drop=True)
# replace the wrong name in the dataframe
df_final['Neighborhood']=df_final['Neighborhood'].str.replace('Beaumond Heights ' , 'Beaumonde Heights')
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.754326,-79.449117
1,M4A,North York,Victoria Village,43.754326,-79.449117
2,M5A,Downtown Toronto,Regent Park,43.656322,-79.380916
3,M5A,Downtown Toronto,Harbourfront,43.656322,-79.380916
4,M6A,North York,Lawrence Manor,43.754326,-79.449117
...,...,...,...,...,...
197,M8Z,Etobicoke,Mimico NW,43.643556,-79.565633
198,M8Z,Etobicoke,The Queensway West,43.643556,-79.565633
199,M8Z,Etobicoke,South of Bloor,43.643556,-79.565633
200,M8Z,Etobicoke,Kingsway Park South West,43.643556,-79.565633


get the latitude and longitude of each neighborhood

In [24]:
neighborhood_list=df_final['Neighborhood'].values
Latitude=[]
Longitude=[] 
for each_neighborhood in neighborhood_list:
    #print(each_neighborhood)
    try:
        geolocator = Nominatim(user_agent="toronto_explorer")
        g=geolocator.geocode('{}, Toronto, Ontario'.format(each_neighborhood))
        #print("g is {}".format(g) )
                     
        Latitude.append(g.latitude)
        Longitude.append(g.longitude)
    except:
        Latitude.append(0)
        Longitude.append(0)
        
#print(Latitude,Longitude)

df_final['Latitude']=Latitude
df_final['Longitude']=Longitude
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
3,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
4,M6A,North York,Lawrence Manor,43.722079,-79.437507
...,...,...,...,...,...
197,M8Z,Etobicoke,Mimico NW,43.616677,-79.496805
198,M8Z,Etobicoke,The Queensway West,43.623618,-79.514764
199,M8Z,Etobicoke,South of Bloor,43.667662,-79.394698
200,M8Z,Etobicoke,Kingsway Park South West,43.647381,-79.511333


clean the data, remove the neighborhood with incorrect lat and lng

In [25]:
df_final[df_final['Latitude']==0]


,PostalCode,Borough,Neighborhood,Latitude,Longitude
6,M7A,Queen's Park,Ontario Provincial Government,0.0,0.0
13,M5B,Downtown Toronto,"Garden District, Ryerson",0.0,0.0
36,M6E,York,Caledonia-Fairbanks,0.0,0.0
100,M6M,York,Keelsdale and Silverthorn,0.0,0.0
166,M8V,Etobicoke,Humber Bay Shores,0.0,0.0


In [26]:
df_final=df_final[df_final['Latitude']!=0]
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
3,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
4,M6A,North York,Lawrence Manor,43.722079,-79.437507
...,...,...,...,...,...
197,M8Z,Etobicoke,Mimico NW,43.616677,-79.496805
198,M8Z,Etobicoke,The Queensway West,43.623618,-79.514764
199,M8Z,Etobicoke,South of Bloor,43.667662,-79.394698
200,M8Z,Etobicoke,Kingsway Park South West,43.647381,-79.511333
